<a href="https://colab.research.google.com/github/theKirill/RecognitionOfRoadSigns/blob/master/CourseWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
import numpy as np

In [0]:
from skimage import io, transform

In [0]:
import matplotlib.pyplot as plt

In [0]:
import pandas as pd

In [0]:
from PIL import Image, ImageDraw

In [0]:
import cv2

In [0]:
import keras 

In [0]:
from keras.models import Sequential 

In [0]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [0]:
from keras.utils import to_categorical

In [0]:
from keras.callbacks import ModelCheckpoint

In [0]:
from keras.models import load_model

In [0]:
class RoadSignLocalization:
  def __init__(self):
    self.__path_images = "drive/My Drive/Images_Train/" #путь, где хранятся изображения
    self.__extension_images = ".ppm" #расширения изображений
    self.__IMG_SIZE = 32 #размер изображения
    self.__images_of_signs = [] # Массив для хранения всех изображений знаков
    self.__classes_of_signs = [] # Массив для хранения номера класса каждого знака
    
  @property
  def path_images(self):
    return self.__path_images

  @path_images.setter
  def path_images(self, path):
    self.__path_images = path

  @property
  def extension_images(self):
    return self.__path_images

  @path_images.setter
  def extension_images(self, extension):
    self.__extension_images = extension

  @property
  def images_of_signs(self):
    return np.array(self.__images_of_signs, dtype = "float32") / 255.0

  @property
  def classes_of_signs(self):
    return to_categorical(self.__classes_of_signs, CLASSES_COUNT)

  #"Вырезаем" область знака
  def get_sign_area(name_img, y_left, y_right, x_left, x_right):
    img = cv2.imread(name_img)
    # Обрезаем изображение по прямоугольнику
    area_sign = img[y_left:y_right, x_left:x_right]
    cv2.imwrite(name_img[:-4] + "_new.ppm", area_sign)

  #Масштабируем изображение до 32*32
  def scale_img(self, name_img):
    basewidth = 300
    img = Image.open(name_img)

    draw = ImageDraw.Draw(img)  # Создаем инструмент для рисования.
    width = img.size[0]  # Определяем ширину.
    height = img.size[1]  # Определяем высоту.
    pix = img.load()  # Выгружаем значения пикселей.

    wpercent = (basewidth / float(img.size[0]))
    # Масштабируем до нужного размера
    img = img.resize((self.__IMG_SIZE, self.__IMG_SIZE), Image.ANTIALIAS)
    img.save(name_img)

  # Получение всех содержащихся обучающих изображений знаков и номеров классов этих знаков
  def get_all_train_images(self):
    for root, dirs, files in os.walk(self.__path_images):
      sign_class = root[-2:]
      if sign_class.isdigit():
        name_csv = root + "/GT-000" + sign_class + ".csv"
        file_csv = pd.read_csv(name_csv, delimiter = ';')
        #Берём углы знаков для каждого изображения из csv файла
        x_left = file_csv['Roi.X1']
        y_left = file_csv['Roi.Y1']
        x_right = file_csv['Roi.X2']
        y_right = file_csv['Roi.Y2']
        i = 0
        for file in files: #каждое изображение обрезаем по области знака, масштабируем до 32*32
          if file[-4:] == self.__extension_images:
            get_sign_area(root +'/' + file, y_left[i], y_right[i], x_left[i], x_right[i])
            scale_img(root + '/' + file[:-4] + "_new.ppm")
            self.__images_of_signs.append(io.imread(root + '/' + file[:-4] + "_new.ppm"))#добавляем в массив изображений
            self.__classes_of_signs.append(int(sign_class))#заносим информацию о классе знака
            i+=1

  # Получение всех тестовых изображений знаков и номеров классов этих знаков
  def get_all_test_images(self):
    self.__path_images = "drive/My Drive/Images_Test/"
    self.__images_of_signs = []

    file_csv = pd.read_csv(self.__path_images + 'GT-final_test.csv', delimiter=';') #читаем информацию о каждом изображении
    #Берём углы и классы знаков для каждого изображения из csv файла
    images_names = file_csv['Filename']
    x_left = file_csv['Roi.X1']
    y_left = file_csv['Roi.Y1']
    x_right = file_csv['Roi.X2']
    y_right = file_csv['Roi.Y2']
    self.__classes_of_signs = file_csv["ClassId"]

    for i in range(images_names.size):#каждое изображение обрезаем по области знака, масштабируем до 32*32
      get_sign_area(self.__path_images + images_names[i], y_left[i], y_right[i], x_left[i], x_right[i])
      scale_img(self.__path_images + images_names[i][:-4] + "_new.ppm")
      self.__images_of_signs.append(io.imread(self.__path_images + images_names[i][:-4] + "_new.ppm"))

In [0]:
class RoadSignClassification:
  def __init__(self):
    self.__EPOCHS_COUNT = 50
    self.__BATCH_SIZE = 200
    self.__X_train = []
    self.__y_train = []
    self.__X_test = []
    self.__y_test = []
    self.__history = []

  @property
  def EPOCHS_COUNT(self):
    return self.__EPOCHS_COUNT

  @EPOCHS_COUNT.setter
  def EPOCHS_COUNT(self, count):
    self.__EPOCHS_COUNT = count

  @property
  def BATCH_SIZE(self):
    return self.__BATCH_SIZE

  @BATCH_SIZE.setter
  def BATCH_SIZE(self, size):
    self.__BATCH_SIZE = size

  def get_model():
    model = Sequential() # Создание модели
    model.add(Conv2D(32, (3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 3), padding = 'same', activation = 'relu'))
    
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))

    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3, 3), padding = 'same', activation = 'relu'))
    
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())

    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(CLASSES_COUNT, activation = 'softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  
    return model

  # подготавливаем изображения для обучения
  def prepocess_train_images(self):
    localization = RoadSignLocalization()
    localization.get_all_train_images()
    self.__X_train = localization.images_of_signs
    self.__y_train = localization.classes_of_signs

  # подготавливаем изображения для теста
  def prepocess_test_images(self):
    localization = RoadSignLocalization()
    localization.get_all_test_images()
    self.__X_test = localization.images_of_signs
    self.__y_test = localization.classes_of_signs

  # обучаем модель
  def fit_model(self):
    my_model = get_model()
    self.__history = my_model.fit(self.__X_train, self.__y_train, epochs = self.__EPOCHS_COUNT, batch_size = self.__BATCH_SIZE, validation_split = 0.1,
         callbacks=[ModelCheckpoint('drive/My Drive/Models/modelBest____.h5', save_best_only = True)])

  # получаем ранее обученную модель  
  def get_trained_model():
    model = load_model('drive/My Drive/Models/modelBest03_11(11).h5')
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

  # предсказываем классы
  def predict_classes():
    trained_model = get_trained_model()
    prediction = trained_model.predict_classes(self.__X_test)
    return np.sum(prediction == self.__y_test) / np.size(prediction) 